# Entendimento do Negócio

O poder legistativo é de uma importância para a sociedade e é essencial para o funcionamento de qualquer regime democrático. Sendo um elo entre o povo e os representantes, este poder deve garantir que todos os brasileiros possuam direitos, garantias e deveres, além de suas liberdades. O legislativo é o responsável pela produção de leis que orientam a sociedade e sediar debates de interesse nacional.

Há duas casas presentes no poder legislativo Câmara dos Deputados e Senado Federal. Um inicia o processo legislativo e o outro a revisa, ou seja ambas se manifestam na elaboração das leis. Inicialmente para este projeto vamos focar no âmbito da Câmara dos deputados! Esta possui os representantes elegidos pela população que apresentam pautas legislativas que influenciam diretamente no dia a dia. 

Tendo essa influência em mente, temos que a Câmara é uma das instituições mais importantes do nosso país e impactante para a população brasileira, segmentos empresariais e grupos minoritários. Uma correta transparência do que tramita na Câmara tornando fácil aproximar o que é de interesse de cada indivíduo para acompanhamento pode trazer uma participação mais ativa dos agentes interessados para a fiscalização e pressão ao poder legislativo, evitando a criação e gastos de dinheiro público com propostas lesivas para o povo e planeta.

Este projeto se insere em fornecer uma forma de facilitar a análise do que será discutido na agenda da câmara semanalmente de acordo com um tema específico, fomentando o engajamento e acompanhamento. Para isso iremos usar os dados abertos da agenda e dos eventos que se inserem (podem ser reuniões, audiências ou sessões de plenário) e aplicar técnicas de Processamento de Linguagem Natural e Inteligência Artificial.

A agenda pode ser visualizada no site: https://www.camara.leg.br/agenda

A Câmara dos deputados também fornece uma API para acesso de alguns dados abertos, muitos dados não estão portados para a API ou necessitam de técnicas de Webscrap.

# Coleta e Entendimento dos Dados

In [ ]:
# Bibliotecas adicionais instaladas no ambiente virtual.

# !pip install pandas requests
# !pip install pdfminer.six
# !pip install beautifulsoup4
# !pip install tqdm

Nosso primeiro objetivo é resgatar dados que auxiliem no acompanhamento das informações dos eventos do legislativo.

Algumas perguntas de guia são:
- Como recuperar algo que seja do interesse de determinado tema?
- Como recuperar algo que seja de interesse de outro órgão ou tipo de interessado?

Com isso podemos facilitar a organização e o processo de informação frente a quantia de eventos na agenda, possibilitando gerar alertas caso algo que esteja presente em pauta que impacte os interessados. Posteriormente é possível analisar o quanto determinada pauta ou assunto interfira e seja benéfico para população (de forma subjetiva usando LLMs).

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

from pdfminer.high_level import extract_text
from bs4 import BeautifulSoup

from urllib.parse import urlparse, parse_qs
import os


Vamos criar um filtro para resgatar eventos apenas da próxima semana. Vamos checar se o dia da semana é final de semana, de forma a condicionar de pegar a agenda dessa semana ou da próxima.

In [2]:
today = datetime.today()
weekday = today.weekday()

if weekday < 5:
    start_date = today

    days_until_saturday = 5 - today.weekday() # Sábado é valor 5 
    end_date = today + timedelta(days=days_until_saturday)

elif weekday == 5:
    start_date = today + timedelta(days=1)
    end_date = today + timedelta(days=7)

else:
    start_date = today
    end_date = today + timedelta(days=6)


print(f"Data de início: {start_date.strftime('%Y-%m-%d')}")
print(f"Data de fim: {end_date.strftime('%Y-%m-%d')}")


Data de início: 2025-06-29
Data de fim: 2025-07-05


Utilizando o endpoint da API pública conseguimos pegar os dados iniciais de eventos a partir do filtro fornecido, como podem ser diversos precisamos aplicar uma técnica de paginação.

In [3]:
def get_events(start_date, end_date, itens_per_page=100):
    url = "https://dadosabertos.camara.leg.br/api/v2/eventos"
    headers = {
        "Accept": "application/json"
    }

    page = 1
    events = []
    while True:
        params = {
            "dataInicio": start_date,
            "dataFim": end_date,
            "ordenarPor": "dataHoraInicio",
            "itens": itens_per_page,
            "ordem": "ASC",
            "pagina": page
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Erro ao requisitar página {page}: {response.status_code}")
            break

        data = response.json()["dados"]

        if not data:
            break

        events.extend(data)
        print(f"Página {page} carregada com {len(data)} eventos.")
        page += 1

    return events

In [4]:
events = get_events(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'))

Página 1 carregada com 80 eventos.


In [5]:
events_df = pd.DataFrame(events)
events_df.head()

,id,uri,dataHoraInicio,dataHoraFim,situacao,descricaoTipo,descricao,localExterno,orgaos,localCamara,urlRegistro
0,75916,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T10:00,None,Cancelada,Audiência Pública,Adaptação das Estruturas Físicas das Escolas F...,Plenário a definir,"[{'id': 2009, 'uri': 'https://dadosabertos.cam...","{'nome': None, 'predio': None, 'sala': None, '...",None
1,76160,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T13:00,None,Convocada,Audiência Pública,Planejamento e Diretrizes das ações do Governo...,None,"[{'id': 537480, 'uri': 'https://dadosabertos.c...","{'nome': 'Anexo II, Plenário 13', 'predio': No...",None
2,76192,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-01T13:00,None,Convocada,Seminário,Inclusão de diretrizes para uma educação antir...,None,"[{'id': 539384, 'uri': 'https://dadosabertos.c...","{'nome': 'Anexo II, Plenário 12', 'predio': No...",None
3,76323,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-02T11:00,None,Agendada,Sessão Não Deliberativa Solene,"Homenagem aos 77 anos da Nakba, a catástrofe p...",None,"[{'id': 180, 'uri': 'https://dadosabertos.cama...","{'nome': 'Plenário da Câmara dos Deputados', '...",None
4,76326,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-02T09:00,None,Agendada,Sessão Não Deliberativa Solene,Homenagem aos 20 anos do Instituto Sabin\r\n S...,None,"[{'id': 180, 'uri': 'https://dadosabertos.cama...","{'nome': 'Plenário da Câmara dos Deputados', '...",None


In [7]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              80 non-null     int64 
 1   uri             80 non-null     object
 2   dataHoraInicio  80 non-null     object
 3   dataHoraFim     0 non-null      object
 4   situacao        80 non-null     object
 5   descricaoTipo   80 non-null     object
 6   descricao       80 non-null     object
 7   localExterno    15 non-null     object
 8   orgaos          80 non-null     object
 9   localCamara     80 non-null     object
 10  urlRegistro     0 non-null      object
dtypes: int64(1), object(10)
memory usage: 7.0+ KB


Temos a maioria das colunas com dados do tipo object e com todos os valores não nulos. Todas as colunas são descritivas, ou seja não temos dados de valores numéricos.

- id: ID do Evento
- uri: URL de informações do evento (redireciona para o outro XML da api com detalhes do evento)
- dataHoraInicio: Data estimada para início do evento
- dataHoraFim: Data de finalização, há todos os valores nulos pois provavelmente será só preenchida para eventos finalizados
- Situacao do Evento: Se está confirmado, cancelado, etc.
- descricaoTipo: A descrição do tipo do evento
- descricao: A descrição do evento, resumo do que será discutido
- localExterno: Local externo do evento
- orgaos: A que comissão ou estrutura o evento de refere
- localCamara: o local dentro da câmara que ele irá ocorrer.
- urlRegistro: todos os campos nulos, não descritivo ao que se refere.

Temos que um evento pode estar associado a várias comissões, elas estão dispostas na coluna órgãos. Pode ser interessante realizar um repasse dos dataframes para SQL (isso não será feito em primeiro momento) de forma que possamos pegar informações estatísticas atreladas também as comissões. Não podemos garantir que sempre terá apenas uma comissão associada. 

In [31]:
event_sample = events_df.iloc[2]
event_sample.orgaos

[{'id': 539384,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539384',
  'sigla': 'CPOVOS',
  'nome': 'Comissão da Amazônia e dos Povos Originários e Tradicionais',
  'apelido': 'Amazônia e dos Povos Originários e Tradicionais',
  'codTipoOrgao': 2,
  'tipoOrgao': 'Comissão Permanente',
  'nomePublicacao': 'Comissão da Amazônia e dos Povos Originários e Tradicionais',
  'nomeResumido': 'Amazônia'}]

Temos aqui um exemplo de comissão ao que o evento de refere.

Analisando a URL disposta nos dados, obtemos um XML com informações adicionais e links interessantes.

In [32]:
event_sample.uri

'https://dadosabertos.camara.leg.br/api/v2/eventos/76192'

Vamos pesquisar mais detalhes do evento.

In [33]:
event_detail_url = event_sample.uri

response = requests.get(event_detail_url, headers={"Accept": "application/json"})

if response.status_code == 200:
    event_detail = response.json()
    print(f"Detalhes resgatados: {event_detail}")
else:
    print(f"Erro na requisição de detalhes {response.status_code}")


Detalhes resgatados: {'dados': {'uriDeputados': None, 'uriConvidados': None, 'fases': None, 'id': 76192, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/76192', 'dataHoraInicio': '2025-07-01T13:00', 'dataHoraFim': None, 'situacao': 'Convocada', 'descricaoTipo': 'Seminário', 'descricao': 'Inclusão de diretrizes para uma educação antirracista no Plano Nacional de Educação\r\n REQ nº 23/2025, de autoria da Deputada Dandara.\r\n\r\nConvidados:\r\n\r\nMESA 1 - Saberes Fundantes\r\n\r\nCLEDISSON SANTOS\r\nMinistério da Igualdade Racial - MIR\r\n\r\nCLÉLIA MARA DOS SANTOS\r\nMinistério da Educação - MEC\r\n\r\nEDSON MACHADO DE BRITO (EDSON KAYAPÓ)\r\nInstituto Federal da Bahia - IFBA\r\n\r\nIRANEIDE SOARES DA SILVA\r\nAssociação Brasileira de Pesquisadores Negros - ABPN\r\n\r\nVANDA MACHADO\r\nUniversidade Federal do Recôncavo da Bahia - UFRB\r\n\r\n\r\nMESA 2 - Vozes dos Movimentos Sociais \r\n\r\nROSA NEGRA \r\nMovimento Negro Unificado - MNU\r\n\r\nDOUGLAS BELCHIOR\r\nUneafro Bra

Temos nos detalhes um pouco mais de informações, como por exemplo os Requerimentos e a url para a Pauta do documento, também temos link para os convidados.

In [34]:
event_detail

{'dados': {'uriDeputados': None,
  'uriConvidados': None,
  'fases': None,
  'id': 76192,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/76192',
  'dataHoraInicio': '2025-07-01T13:00',
  'dataHoraFim': None,
  'situacao': 'Convocada',
  'descricaoTipo': 'Seminário',
  'descricao': 'Inclusão de diretrizes para uma educação antirracista no Plano Nacional de Educação\r\n REQ nº 23/2025, de autoria da Deputada Dandara.\r\n\r\nConvidados:\r\n\r\nMESA 1 - Saberes Fundantes\r\n\r\nCLEDISSON SANTOS\r\nMinistério da Igualdade Racial - MIR\r\n\r\nCLÉLIA MARA DOS SANTOS\r\nMinistério da Educação - MEC\r\n\r\nEDSON MACHADO DE BRITO (EDSON KAYAPÓ)\r\nInstituto Federal da Bahia - IFBA\r\n\r\nIRANEIDE SOARES DA SILVA\r\nAssociação Brasileira de Pesquisadores Negros - ABPN\r\n\r\nVANDA MACHADO\r\nUniversidade Federal do Recôncavo da Bahia - UFRB\r\n\r\n\r\nMESA 2 - Vozes dos Movimentos Sociais \r\n\r\nROSA NEGRA \r\nMovimento Negro Unificado - MNU\r\n\r\nDOUGLAS BELCHIOR\r\nUneafro Brasil

A partir daqui temos alguns caminhos interessantes para explorar, como:

- Requerimentos, temos o link do requerimento associado ao evento
- Pautas: Podemos ter um sumário diferente ao que o evento se refere, nas pautas também podemos encontrar os convidades e representantes necessários para o evento.

A informação de convidados pelo que é visto ao acessar os documentos é repetida em ambos, porém alguns eventos não possuem requerimento, apenas pauta.

Essa resposta de detalhes está defasada em alguns pontos:
- Temos que links leva a si mesmo
- 'urlDocumentoPauta' também não é válido e leva a um XML simples que não possui tanta informação. Devido a isso devemos achar outra forma de encontrar a pauta. Pensando nisso adentrei em uma pauta manual pelo site da agenda, podemos realizar um Webscrap com a url base, alterando apenas o código da pauta.

In [35]:
event_id = event_detail["dados"]["id"]
event_url = f"https://www.camara.leg.br/evento-legislativo/{event_id}"
event_url

'https://www.camara.leg.br/evento-legislativo/76192'

Verificando o site da câmara para extrair a pauta temos:
(inserir imagem)


Com isso podemos utilizar o BeautifulSoup para extrair o link necessário de acesso a Pauta.

In [36]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_event = requests.get(event_url, headers=headers)

if response_event.status_code == 200:
    soup_event = BeautifulSoup(response_event.text, "html.parser")
    itens = soup_event.find_all(class_="links-adicionais__item")

    cleaned_links = {}
    for item in itens:
        texto = item.get_text(strip=True)
        link = item.find("a")["href"] if item.find("a") else None

        cleaned_links[texto] = link
        
        print(f"Texto: {texto}")
        print(f"Link: {link}")
else:
    print(f"Erro ao acessar a página: {response.status_code}")

Texto: Requerimento
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2927751
Texto: Documentos da reunião
Link: #documentos-modal
Texto: Pauta
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2895879


Aqui temos por exemplo o link também para o requerimento.

In [37]:
cleaned_links

{'Requerimento': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2927751',
 'Documentos da reunião': '#documentos-modal',
 'Pauta': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2895879'}

In [38]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_topics = requests.get(cleaned_links['Pauta'], headers=headers)
if response_topics.status_code != 200:
    raise Exception(f"Erro {response_topics.status_code}")

soup_topics = BeautifulSoup(response_topics.text, "html.parser")


In [28]:
soup_topics

<!--
   Licensed to the Apache Software Foundation (ASF) under one or more
   contributor license agreements.  See the NOTICE file distributed with
   this work for additional information regarding copyright ownership.
   The ASF licenses this file to You under the Apache License, Version 2.0
   (the "License"); you may not use this file except in compliance with
   the License.  You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
-->
<!-- prolog breaks internet explorer, WICKET-2718 -->
<!-- < ? xml version="1.0" encoding="UTF-8" ? > -->
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/lo

Obtemos que a URL disponível no site é inválida, na realidade ela mostra que é feito um redirecionamento automático para outra URL! Podemos utilizar esta outra URL novamente como uma URL base e alterar apenas o código que direciona para a pauta. URL que é direcionado: https://www.camara.leg.br/internet/ordemdodia/integras/

In [39]:
# Resgatar o ID que estava como query parameter na URL antiga
parsed_url = urlparse(cleaned_links['Pauta'])
query_params = parse_qs(parsed_url.query)

codteor = query_params.get("codteor", [None])[0]

true_topics_url = f"https://www.camara.leg.br/internet/ordemdodia/integras/{codteor}.htm"
true_topics_url

'https://www.camara.leg.br/internet/ordemdodia/integras/2895879.htm'

In [41]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(true_topics_url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Erro {response.status_code}")

soup_true_topics = BeautifulSoup(response.text, "html.parser")


Neste ponto temos que a Pauta possui uma diagramação de HTML muito bagunçada e cheia de divisões! Também não poderia garantir que todas as pautas possuiriam as mesmas áreas e tópicos de texto. Devido a isso decidi resgatar todo o texto como um grande bloco, possível posteriormente pegar "a partir de tal palavra", por hora dessa forma é suficiente.

In [42]:
# Extrai o texto visível da página, sem tags HTML
topics_text = soup_true_topics.get_text(separator="\n", strip=True)

print(topics_text[:2000])

Pauta - CPOVOS - 01/07/2025 13:00
CÂMARA DOS DEPUTADOS
COMISSÃO DA AMAZÔNIA E DOS POVOS ORIGINÁRIOS E TRADICIONAIS
57ª Legislatura - 3ª Sessão Legislativa Ordinária
PAUTA DE
SEMINÁRIO
DIA 01/07/2025
LOCAL:
Anexo II, Plenário 12
HORÁRIO:
13h
TEMA:
"Inclusão de diretrizes para uma educação antirracista no Plano Nacional de Educação"
REQ nº 23/2025, de autoria da Deputada Dandara.
Convidados:
MESA 1 - Saberes Fundantes
CLEDISSON SANTOS
Ministério da Igualdade Racial - MIR
CLÉLIA MARA DOS SANTOS
Ministério da Educação - MEC
EDSON MACHADO DE BRITO (EDSON KAYAPÓ)
Instituto Federal da Bahia - IFBA
IRANEIDE SOARES DA SILVA
Associação Brasileira de Pesquisadores Negros - ABPN
VANDA MACHADO
Universidade Federal do Recôncavo da Bahia - UFRB
MESA 2 - Vozes dos Movimentos Sociais
ROSA NEGRA
Movimento Negro Unificado - MNU
DOUGLAS BELCHIOR
Uneafro Brasil
AFONSO GOMES
Coletivo Nacional da Juventude Negra - Enegrecer
THALES VIEIRA - presença confirmada
Observatório da Branquitude
IÊDA LEAL
Confederaçã

---

Partindo para os requerimentos, temos que nas informações de detalhes do evento há URIs funcionais para acessá-los, a url direciona para um XML de detalhes e neste XML temos o caminho para o documento inteiro. Vamos resgatá-lo para salvarmos todo o texto e fazermos análise textual. Há a possibilidade de criarmos outro DataFrame com informações apenas dos requerimentos, uma vez que um evento pode ter vários.

In [43]:
motion_url = event_detail['dados']['requerimentos'][0]['uri']

response_motion = requests.get(motion_url, headers={"Accept": "application/json"})

if response_motion.status_code == 200:
    print(response_motion.json())

motion_detail = response_motion.json()

motion_detail

{'dados': {'id': 2520836, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2520836', 'siglaTipo': 'REQ', 'codTipo': 686, 'numero': 23, 'ano': 2025, 'ementa': 'Requer a realização de Seminário para debater a inclusão de diretrizes para uma educação antirracista no Plano Nacional de Educação.', 'dataApresentacao': '2025-06-06T12:28', 'uriOrgaoNumerador': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539384', 'statusProposicao': {'dataHora': '2025-06-17T13:00', 'sequencia': 4, 'siglaOrgao': 'CPOVOS', 'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539384', 'uriUltimoRelator': None, 'regime': '.', 'descricaoTramitacao': 'Aprovação de Proposição Interna', 'codTipoTramitacao': '241', 'descricaoSituacao': 'Aguardando Providências Internas', 'codSituacao': 936, 'despacho': 'Aprovado o Requerimento.', 'url': None, 'ambito': 'Regimental', 'apreciacao': 'Indefinida'}, 'uriAutores': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2520836/autores', 'descricaoTi

{'dados': {'id': 2520836,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2520836',
  'siglaTipo': 'REQ',
  'codTipo': 686,
  'numero': 23,
  'ano': 2025,
  'ementa': 'Requer a realização de Seminário para debater a inclusão de diretrizes para uma educação antirracista no Plano Nacional de Educação.',
  'dataApresentacao': '2025-06-06T12:28',
  'uriOrgaoNumerador': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539384',
  'statusProposicao': {'dataHora': '2025-06-17T13:00',
   'sequencia': 4,
   'siglaOrgao': 'CPOVOS',
   'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539384',
   'uriUltimoRelator': None,
   'regime': '.',
   'descricaoTramitacao': 'Aprovação de Proposição Interna',
   'codTipoTramitacao': '241',
   'descricaoSituacao': 'Aguardando Providências Internas',
   'codSituacao': 936,
   'despacho': 'Aprovado o Requerimento.',
   'url': None,
   'ambito': 'Regimental',
   'apreciacao': 'Indefinida'},
  'uriAutores': 'https://dadosabertos.cama

Do requerimento temos de interessante:  Ementa, Autores e URL de inteiro Teor.

Até então temos a estrutura:
- Evento
  - Pode possuir vários requerimentos
    - Um requerimento pode possuir vários autores.
    - Possui uma pauta

Diferente das pautas que estão em HTML os documentos são lidos como PDF, ou seja, precisamos baixá-los e depois convertemos para um texto tratável.

In [44]:
import requests

motions_folder = '../data/motions/'

motion_pdf_url = motion_detail['dados']['urlInteiroTeor']

headers = {
    "User-Agent": "Mozilla/5.0",  # finge ser um navegador
    "Accept": "application/pdf"
}

response_motion_pdf = requests.get(motion_pdf_url, headers=headers)

motion_number = motion_detail['dados']['numero']
motion_year = motion_detail['dados']['ano']

pdf_name = os.path.join(motions_folder, f"{motion_number}_{motion_year}.pdf")

if response_motion_pdf.status_code == 200:
    with open(pdf_name, "wb") as f:
        f.write(response_motion_pdf.content)
    print("PDF baixado com sucesso.")
else:
    print(f"Erro {response_motion_pdf.status_code}")


PDF baixado com sucesso.


Vamos testar extrair o texto de um único documento.

In [45]:
texto = extract_text(pdf_name)

# Exibe os primeiros caracteres
print(texto[:1000])

CÂMARA DOS DEPUTADOS
COMISSÃO DA AMAZÔNIA E DOS POVOS ORIGINÁRIOS E TRADICIONAIS 

REQUERIMENTO Nº .............., DE 2025
(Da Sra. Dandara)

Requer a realização de Seminário
para   debater   a   inclusão   de
diretrizes   para   uma   educação
antirracista no Plano Nacional de
Educação.

Senhora Presidente,

Requeiro a Vossa Excelência, ouvido o Plenário desta Comissão, nos termos
regimentais,   a   realização   de  Seminário,   no   âmbito   desta   Comissão,   em
parceria com a  Frente Parlamentar Mista Antirracismo, com o objetivo de
debater  a inclusão de  diretrizes  para  uma  educação antirracista  no  Plano
Nacional   de   Educação   (2024–2034),   em   data   a   ser   definida,   tendo   como
marco o lançamento oficial do  Caderno PNE Antirracista, com os seguintes
convidados.

Convidados/as:

 Petronilha Beatriz Gonçalves e Silva  – Universidade Federal de São

Carlos (UFSCAR)

 Edson Machado de Brito (Edson Kayapó) – Instituto Federal da Bahia



(IFBA)
Iraneide Soares 

Com isso temos uma estrutura de resgate dos dados que desejamos! Para melhor Organizar a criação de dataframes, vamos repassar os códigos aqui desenvolvidos para scripts mais funcionais e organizados. Presentes na pasta `src`.

Porém agora temos:
- Eventos com detalhes gerais e Pautas associadas com textos extraídos.
- Requerimentos com PDFs e textos extraídos.


Após extrairmos as informações com os scripts iremos realizar a etapa de EDA em um notebook separado. Como há um processamento de PDFs e múltiplos arquivos. Iremos primeiro gerar um CSV com esses dados para depois apenas explorarmos.